In [2]:
import pandas as pd

### Задание 7

In [71]:

dfe = pd.read_excel('../data/документация/КТ_Кировский.xlsx')
dfc = pd.read_csv('../data/документация/sp.csv')

dfec = dfe.copy(deep=True)
dfcc = dfc.copy(deep=True)

dfcc['Сумма'] = dfcc['Сумма'].replace(',', '.', regex=True).astype(float)
dfcc['Пени'] = dfcc['Пени'].replace(',', '.', regex=True).astype(float)
dfcc['ГосПошлина'] = dfcc['ГосПошлина'].replace(',', '.', regex=True).astype(float)

#dfcc['Общий долг'] = dfcc[['Сумма', 'Пени', 'ГосПошлина']].sum(axis=1)

In [178]:
df = dfcc.copy(deep=True)
df['Номер ИД'] = df['Номер ИД'].str.split(r',\s*')
df = df.explode('Номер ИД')

df[df['Номер ИД'] == "2-88-1941/2022"], df[df['Номер ИД'] == "2-88-1942/2022"]


(              Номер ИД       Номер       Счет      Сумма     Пени  ГосПошлина
 377816  2-88-1941/2022  2148135485  3278559.0  128377.34  2755.62         0.0
 377817  2-88-1941/2022  2148135485  3278559.0  128377.34  9439.56      1023.2
 377818  2-88-1941/2022  2148135485  3278559.0  128377.34  9439.56      1023.2,
               Номер ИД       Номер       Счет      Сумма     Пени  ГосПошлина
 377818  2-88-1942/2022  2148135485  3278559.0  128377.34  9439.56      1023.2)

In [107]:
merged_df = pd.merge(dfec, dfcc[['Номер ИД', 'Сумма', 'Пени', 'ГосПошлина']], how='left').drop_duplicates()
merged_df['Общий долг'] = merged_df[['Сумма', 'Пени', 'ГосПошлина']].sum(axis=1)

print(f"Null - {merged_df[merged_df['Общий долг'].isna()].shape}, All - {merged_df.shape}")


Null - (0, 35), All - (2520, 35)


In [ ]:
pd.set_option('display.max_columns', None)
merged_df[merged_df['Общий долг'] == 0]

In [108]:
merged_df.to_excel('../data/документация/КТ_Кировский_ОбщийДолг.xlsx')

In [72]:
dfcc.dtypes

Номер ИД       object
Номер           int64
Счет          float64
Сумма         float64
Пени          float64
ГосПошлина    float64
dtype: object

### Задание 8 - Общий отчет

In [ ]:
fz = pd.read_csv("../data/документация/fz.csv")
sv = pd.read_excel("../data/документация/Стадии взысканий.xlsx")

fzcopy = fz.copy(deep=True)
fzcopy = pd.merge(fzcopy, sv, how="left")


fzcopy['Сумма'] = pd.to_numeric(fzcopy['Сумма'].replace(',', '.', regex=True).astype(float))
fzcopy['ROW_ID_Взыскания'] = pd.to_numeric(fzcopy['ROW_ID_Взыскания'], errors='coerce').fillna(0).astype(int)
fzcopy['НомерДела'] = pd.to_numeric(fzcopy['НомерДела'], errors='coerce').fillna(0).astype(int)
fzcopy['Месяц долга'] = pd.to_datetime(fzcopy['Месяц долга'])


In [ ]:
fzcopy.dtypes

Сумма                             float64
Номер ЛЦ                            int64
Счет-Наниматель                   float64
Месяц долга                datetime64[ns]
Наименование                       object
ROW_ID_Организации                  int64
ФИО                                object
Дата рождения                      object
Дата смерти                        object
ROW_ID_Взыскания                    int64
Фаза                               object
Дата постановки на фазу            object
ДатНачДолга                        object
ДатКнцДолга                        object
НомерДела                           int64
Общая стадия                       object
Сомнительность                     object
dtype: object

In [ ]:
def Period(df):
    
    period = pd.to_datetime('2025-02-01')

    df['months_diff'] = (
        (period.year - df['Месяц долга'].dt.year) * 12
        + (period.month - df['Месяц долга'].dt.month)
    )

    df['years_diff'] = df['months_diff'] // 12

    
    def create_row(aLS, aNS, sum):
        return pd.DataFrame([{'КоличествоЛЦ': aLS, 'КоличествоДел': aNS, 'СуммаДолга': sum}])

    dfp = pd.DataFrame(columns=['КоличествоЛЦ', 'КоличествоДел', 'СуммаДолга'])

    # Отчетный месяц
    dfp = pd.concat([dfp, create_row(
                                    df.query('months_diff == 0')['Номер ЛЦ'].nunique(),
                                    df.query('months_diff == 0')['ROW_ID_Взыскания'].nunique(),
                                    df.query('months_diff == 0')['Сумма'].sum())])

    # от 2 до 3 месяцев после отчетного
    dfp = pd.concat([dfp, create_row(
                                    df.query('3 > months_diff >= 2')['Номер ЛЦ'].nunique(),
                                    df.query('3 > months_diff >= 2')['ROW_ID_Взыскания'].nunique(),
                                    df.query('3 > months_diff >= 2')['Сумма'].sum())])

    # от 3 месяцев до 3 лет после отчетного
    dfp = pd.concat([dfp, create_row(
                                    df.query('months_diff >= 3 and years_diff <= 3')['Номер ЛЦ'].nunique(),
                                    df.query('months_diff >= 3 and years_diff <= 3')['ROW_ID_Взыскания'].nunique(),
                                    df.query('months_diff >= 3 and years_diff <= 3')['Сумма'].sum())])

    # свыше 3 лет
    dfp = pd.concat([dfp, create_row(
                                    df.query('years_diff >= 3')['Номер ЛЦ'].nunique(),
                                    df.query('years_diff >= 3')['ROW_ID_Взыскания'].nunique(),
                                    df.query('years_diff >= 3')['Сумма'].sum())])

    return dfp

In [128]:
fzcopy[(fzcopy['Месяц долга'] == pd.to_datetime('2024-10-02')) & fzcopy['Общая стадия'] == 'Суд']

,Сумма,Номер ЛЦ,Счет-Наниматель,Месяц долга,Наименование,ROW_ID_Организации,ФИО,Дата рождения,Дата смерти,ROW_ID_Взыскания,Фаза,Дата постановки на фазу,ДатНачДолга,ДатКнцДолга,НомерДела,Общая стадия,Сомнительность


#### Досудебная фаза

In [145]:
fzDosud = fzcopy[(fzcopy["Общая стадия"] == "Досубедная от месяца") | fzcopy["Фаза"].isna() | fzcopy["Общая стадия"].isna()]


In [ ]:
otchetDosud = Period(fzDosud)
otchetDosud

#### Судебная фаза

In [131]:
fzSud = fzcopy[fzcopy["Общая стадия"] == "Суд"]

In [ ]:
otchetSud = Period(fzSud)
otchetSud

#### ФССП фаза

In [133]:
fzFssp = fzcopy[(fzcopy["Общая стадия"] == "ФССП")]

#### Итог

In [ ]:
# data = [["Отчетный месяц"], ["от 2 до 3 месяцев после отчетного"], ["от 3 месяцев до 3 лет после отчетного"], ["свыше 3 лет"]]
# total_ost = pd.DataFrame(columns="Категория")
# total_ost['Категория']
total_ost = pd.concat([Period(fzDosud), Period(fzSud), Period(fzFssp)], axis=1)
total_ost.to_excel("./total_ost.xlsx")


In [ ]:
total_ces = pd.concat([Period(fzDosud), Period(fzSud), Period(fzFssp)], axis=1)
total_ces.to_excel("./total_ces.xlsx")

In [ ]:
total_ces = pd.concat([Period(fzDosud), Period(fzSud), Period(fzFssp)], axis=1)
total_ces.to_excel("./total_mun.xlsx")

In [192]:
def Category(df):
    sv = pd.read_excel("../data/документация/Стадии взысканий.xlsx")

    dfcopy = df.copy(deep=True)
    dfcopy = pd.merge(dfcopy, sv, how="left")


    dfcopy['Сумма'] = pd.to_numeric(dfcopy['Сумма'].replace(',', '.', regex=True).astype(float))
    dfcopy['ROW_ID_Взыскания'] = pd.to_numeric(dfcopy['ROW_ID_Взыскания'], errors='coerce').fillna(0).astype(int)
    dfcopy['НомерДела'] = pd.to_numeric(dfcopy['НомерДела'], errors='coerce').fillna(0).astype(int)
    dfcopy['Месяц долга'] = pd.to_datetime(dfcopy['Месяц долга'])

    # Досудебная
    dfDosud = dfcopy[(dfcopy["Общая стадия"] == "Досубедная от месяца") | dfcopy["Фаза"].isna() | dfcopy["Общая стадия"].isna()]

    # Судебная
    dfSud = dfcopy[dfcopy["Общая стадия"] == "Суд"]

    # ФССП
    dfFssp = dfcopy[(dfcopy["Общая стадия"] == "ФССП")]

    total_df = pd.concat([Period(dfDosud), Period(dfSud), Period(dfFssp)], axis=1)

    return total_df
    

In [188]:
df_ost = pd.read_csv("../data/документация/ost.csv")
df_ces = pd.read_csv("../data/документация/ces.csv")
df_mun = pd.read_csv("../data/документация/mun.csv")

In [202]:
# Остальные
Category(df_ost)

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will n

,КоличествоЛЦ,КоличествоДел,СуммаДолга,КоличествоЛЦ,КоличествоДел,СуммаДолга,КоличествоЛЦ,КоличествоДел,СуммаДолга
0,0,0,0.000000e+00,0,0,0.000000e+00,0,0,0.000000e+00
0,34520,1778,1.026123e+08,1,1,3.354320e+03,0,0,0.000000e+00
0,41349,10298,3.322511e+08,13444,17315,3.001866e+08,14939,32940,4.991188e+08
0,10356,2942,7.038945e+07,6776,7386,1.474535e+08,10886,19384,3.253173e+08


In [201]:
Category(df_ost).to_excel ("../data/документация/total_ost.xlsx")

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will n

In [204]:
# Цессии
Category(df_ces)

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will n

,КоличествоЛЦ,КоличествоДел,СуммаДолга,КоличествоЛЦ,КоличествоДел,СуммаДолга,КоличествоЛЦ,КоличествоДел,СуммаДолга
0,0,0,0.000000e+00,0,0,0.00,0,0,0.00
0,1,1,1.930000e+01,0,0,0.00,0,0,0.00
0,1007,388,1.363419e+07,275,289,1938224.73,847,1464,8963367.00
0,15292,1149,2.052686e+08,850,895,9321051.74,6353,11290,85378344.18


In [206]:
Category(df_ces).to_excel ("../data/документация/total_ces.xlsx")

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will n

In [198]:
# Мунжилье
Category(df_mun)

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will n

,КоличествоЛЦ,КоличествоДел,СуммаДолга,КоличествоЛЦ,КоличествоДел,СуммаДолга,КоличествоЛЦ,КоличествоДел,СуммаДолга
0,0,0,0.00,0,0,0.00,0,0,0.00
0,56,3,156703.38,0,0,0.00,0,0,0.00
0,60,6,424256.82,14,17,323895.28,11,25,472179.71
0,3,1,8897.39,5,5,127954.37,8,14,388659.78


In [203]:
Category(df_mun).to_excel ("../data/документация/total_mun.xlsx")

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38576\4286576348.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will n